In [ ]:
from utils import load_api_key
API_KEY = load_api_key()

In [ ]:
#!pip install colorama -q

# Objetivo: Enfrentar dos LLMs

In [ ]:
from textwrap import dedent  # Útil cuando trabajéis con archivos .py

from colorama import Fore, Style, init

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder  # Nuevo

from langchain_community.chat_message_histories import ChatMessageHistory  # Nuevo

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", api_key=API_KEY)

In [ ]:
SYSTEM_PROMPT = dedent("""
    Eres {name} y debes comportarte como tal tanto en tu estilo de escritura como en tus opiniones.
    Sé siempre consecuente con tu personaje en todos los debates en los que participes.
    Nunca te salgas del personaje, bajo ningún concepto.
    Debes defender tu postura siempre a no ser que tu rival tenga un argumento mejor que el tuyo.
    En ese caso, dale la razón y da por terminado el debate. Tus intervenciones no pueden ser superiores a 5 líneas.
    
    El tema de debate es el siguiente: {topic}
""")

In [ ]:
class Character:
    def __init__(self, name: str):
        self.name = name
        self.chat_history = ChatMessageHistory()

    def answer(self, msg: str, topic: str, color_fmt: str = Fore.RED):
        # Guarda las respuestas del otro interlocutor
        # Este if solamente NO se cumple con el primer mensaje
        if msg:
            self.chat_history.add_user_message(HumanMessage(msg))

        prompt_template = ChatPromptTemplate.from_messages([
            ("system", SYSTEM_PROMPT),
            MessagesPlaceholder("messages")     # Placeholder donde se guardarán la lista de mensajes "messages"
        ])

        chain = prompt_template | llm | StrOutputParser()

        total_msg = ""
        print(f"\n\n{color_fmt}{self.name}: {Style.RESET_ALL}\n\n")

        for chunk in chain.stream({"name": self.name, "topic": topic, "messages": self.chat_history.messages}):
            print(f"{color_fmt}{chunk}{Style.RESET_ALL}", end="", flush=True)
            total_msg += chunk

        return total_msg

In [ ]:
class DialogueSimulator:
    def __init__(self, character1: Character, character2: Character):
        self.character1 = character1
        self.character2 = character2

    def run(self, topic: str, rounds: int = 3):
        output = self.character1.answer(msg="", topic=topic)

        for _ in range(rounds):
            output = self.character2.answer(msg=output, topic=topic, color_fmt=Fore.GREEN)
            output = self.character1.answer(msg=output, topic=topic)

In [ ]:
character_1 = Character(name="Albert Einstein")
character_2 = Character(name="Winston Churchill")

dialogue_simulator = DialogueSimulator(character_1, character_2)

dialogue_simulator.run(topic="La mejor comida del mundo", rounds=2)